## brand

---

In [1]:
import os
import pandas as pd
import requests
import json
import warnings
import sys
sys.path.append('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data_collection/notiAPI')

from auth import NotiServerAuth
from api import NotiServerRequest

from dataclasses import dataclass, field
from typing import Optional, List
from urllib.parse import urlparse, parse_qs

from pprint import pprint

warnings.filterwarnings(action='ignore')

In [2]:
# 최근 10일치 spec 데이터
df = pd.read_csv('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data/spec.csv') 
len(df)

86161

In [3]:
df_test = df.head(20)

In [4]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
brands = []

# 각 아이템에 대한 정보를 조회합니다.
# 브랜드 정보 조회후 브랜드 데이터가 조회되지 않는다면, title에서 brand를 가져옴
for item_id in item_ids:
    res = NotiServerRequest.product_info(ids=[item_id], pivs=[])
    try :
        brand = res.json().get('data', {}).get('products', [])[0]['details']['brand']
    except KeyError:
        try:
            brand = res.json().get('data', {}).get('products', [])[0]['title'].split()[0]
        except :
            brand = 'unknown'
    except:
        brand = 'unknown'

    brands.append(brand)

# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['brand'] = brands
df_test

,user,item,time,brand
0,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663647,MSI
1,e44ba8873d8672b0b427a482996a2de0,97d43f0d5f145be54c0b9f9ad8097477dcc350e04c6234...,1707663652,로지텍
2,2570acc346ff78db06d55da8bd64ecf7,92b1656875d3f4728c0f392ab84e1b54719e80118e3bb1...,1707663653,신일
3,f5805244f125cf7fe6a61cdc1eecb4f7,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663669,MSI
4,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663677,레노버
5,f5805244f125cf7fe6a61cdc1eecb4f7,ef83695f93fa21561692d6910519d45e30f6bf7422a56b...,1707663681,LG전자
6,f6fb3002c49d606dad83b8a96389dee2,a5b557d3325970ec192ab54ad3b4f2eda40903f54a5baf...,1707663684,레노버
7,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663685,MSI
8,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663686,레노버
9,e44ba8873d8672b0b427a482996a2de0,a80ec774d3015f5faee915bb81490b0f4c250a7e9ce3ce...,1707663689,홈플래닛


---

## cate

In [5]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
cates = []

# 각 아이템에 대한 정보를 조회합니다.
for item_id in item_ids:
    try:
        res = NotiServerRequest.product_info(ids=[item_id], pivs=[])
        # 'details'에 'brand' 정보가 있는지 확인하고, 없는 경우 'title' 정보로 대체합니다.
        cate = res.json().get('data', {}).get('products', [])[0]['details']['category_small']
    except KeyError:
        try :
            cate = res.json().get('data', {}).get('products', [])[0]['title'].split(",")[0]
        except :
            cate = 'unknown'
    except :
        cate = 'unknown'
    
    cates.append(cate)

        
# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['cate'] = cates
df_test

,user,item,time,brand,cate
0,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663647,MSI,노트북
1,e44ba8873d8672b0b427a482996a2de0,97d43f0d5f145be54c0b9f9ad8097477dcc350e04c6234...,1707663652,로지텍,마우스
2,2570acc346ff78db06d55da8bd64ecf7,92b1656875d3f4728c0f392ab84e1b54719e80118e3bb1...,1707663653,신일,전기주전자
3,f5805244f125cf7fe6a61cdc1eecb4f7,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663669,MSI,노트북
4,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663677,레노버,레노버 2019 ThinkPad T495 14
5,f5805244f125cf7fe6a61cdc1eecb4f7,ef83695f93fa21561692d6910519d45e30f6bf7422a56b...,1707663681,LG전자,양문형냉장고
6,f6fb3002c49d606dad83b8a96389dee2,a5b557d3325970ec192ab54ad3b4f2eda40903f54a5baf...,1707663684,레노버,노트북
7,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663685,MSI,노트북
8,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663686,레노버,레노버 2019 ThinkPad T495 14
9,e44ba8873d8672b0b427a482996a2de0,a80ec774d3015f5faee915bb81490b0f4c250a7e9ce3ce...,1707663689,홈플래닛,TV


---

## cate 2

In [6]:
# 카테고리별 키워드 정의
category_keywords = {
    '노트북': ['노트북','맥북', '그램','ThinkPad','FLEX','스트릭스','스워드','Zenbook','오멘','블레이드','에일리언웨어', '갤럭시북', '아이디어패드','울트라PC','코어i7', '코어i5', '코어i9','라이젠'],
    '스마트폰': ['스마트폰'],
    '헤드폰': ['헤드폰','헤드셋'],
    '스피커': ['스피커'],
    '모니터': ['모니터'],
    '냉장고': ['냉장고'],
    '마우스': ['마우스', 'G PRO'],
    '청소기': ['청소기'],
    '키보드': ['키보드','키크론'],
    'TV': ['TV','HD'],
    '세탁기': ['세탁기'],         
    '가전/디지털': ['가전/디지털'],  
    '로봇청소기': ['로봇청소기'],
    '게이밍': ['게이밍'],
    '이어폰': ['이어폰'],
    '저장장치': ['저장장치'],
    'unknown' : ['unknown']
}

# 분류를 위한 함수 정의
def categorize_by_keywords(item, category_keywords):
    for category, keywords in category_keywords.items():
        if any(keyword in item for keyword in keywords):
            return category
    return item  # 키워드가 일치하지 않는 경우 etc 처리

# 'cate' 열에 함수 적용
df_test['cate'] = df_test['cate'].apply(lambda x: categorize_by_keywords(x, category_keywords))
df_test

,user,item,time,brand,cate
0,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663647,MSI,노트북
1,e44ba8873d8672b0b427a482996a2de0,97d43f0d5f145be54c0b9f9ad8097477dcc350e04c6234...,1707663652,로지텍,마우스
2,2570acc346ff78db06d55da8bd64ecf7,92b1656875d3f4728c0f392ab84e1b54719e80118e3bb1...,1707663653,신일,전기주전자
3,f5805244f125cf7fe6a61cdc1eecb4f7,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663669,MSI,노트북
4,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663677,레노버,노트북
5,f5805244f125cf7fe6a61cdc1eecb4f7,ef83695f93fa21561692d6910519d45e30f6bf7422a56b...,1707663681,LG전자,냉장고
6,f6fb3002c49d606dad83b8a96389dee2,a5b557d3325970ec192ab54ad3b4f2eda40903f54a5baf...,1707663684,레노버,노트북
7,d09a238c4d508cafa7b9682ddc925031,700e9e85dc36d1630dac894908146a413c7692c575e27b...,1707663685,MSI,노트북
8,f5805244f125cf7fe6a61cdc1eecb4f7,f3cc4234ae225775861ca4fbd9385b29b4dc443404c7ea...,1707663686,레노버,노트북
9,e44ba8873d8672b0b427a482996a2de0,a80ec774d3015f5faee915bb81490b0f4c250a7e9ce3ce...,1707663689,홈플래닛,TV


---